In [1]:
from powersddp.system import PowerSystem
import cvxopt.modeling as model

data = {'load': [50, 50, 50],
        'discretizations': 3,
        'stages': 3,
        'scenarios': 2,
        'shedding_cost': 500,
        'hydro-units': [{'name': 'HU1',
                         'v_max': 100,
                         'v_min': 20,
                         'prod': 0.95,
                         'flow_max': 60,
                         'inflow_scenarios': [[23, 16], [19, 14], [15, 11]]}],
        'thermal-units': [{'name': 'GT1', 'capacity': 15, 'cost': 10},
                          {'name': 'GT2', 'capacity': 10, 'cost': 25}]}

TestSystem = PowerSystem(path='system.yml')

System loaded from system.yml file


## Decision Variables

### Hydro Units
- Final volume $v_f$: The final volume of the reservoir after the operational period
- Turbined flow  $v_t$: The ammount of water that was turbined during the period
- Shed volume $v_v$: The ammount of water that was shed in the period
- Initial volume $v_i$
- Influx $afl$

### Thermal Units
- total generation $g_t$: The total amount of generation provided by the unit during the period

### System
- Outage $def$: The total ammount of power that will not be delivered by the system



## The Objective Function

Assuming a problem with 3 generation units (2 TGUs and 1 HGU) let's write down the Objetive Function of our problem:

$$
\begin{equation}
    \begin{aligned}
        \min \quad                    & C_1\cdot g_{t_1} + C_2\cdot g_{t_2} + C_{def}\cdot def + 0.01\cdot v_v\\
        \textrm{s.t.} \quad           & \\
        \textrm{hydro balance} \quad  & v_f = v_i + afl - v_t \\
        \textrm{load supplying} \quad & \rho\cdot v_t + g_{t_1} + g_{t_2} + def = \textrm{load}\\
        \textrm{constraints} \quad    & \\
                                      & v_{f_{min}}\leq v_f \leq v_{f_{max}}\\
                                      & v_{t_{min}}\leq v_t \leq v_{t_{max}}\\
                                      & v_{v_{min}}\leq v_v \leq v_{v_{max}}\\
                                      & g_{t_{min}}\leq g_t^\ast \leq g_{t_{max}}\\
        ^\ast \textrm{for each TGU}& 
    \end{aligned}
\end{equation}
$$

> Later we shall also add the Future Cost Function $\alpha$ in the minimization function 

In [4]:
def dispatch(system, v_i, inflow):
    n_tgu = len(system.data['thermal-units'])
    n_hgu = len(system.data['hydro-units'])


    ## Initializing Model Variables
    v_f = model.variable(n_hgu, "Final Volume of the Hydro Unit")
    v_t = model.variable(n_hgu, "Turbined Flow of the Hydro Unit")
    v_v = model.variable(n_hgu, "Shed flow of the Hydro Unit")
    g_t = model.variable(n_tgu, "Power generated by the Thermal Unit")
    deficit = model.variable(1, "Power deficit")

    ## Objective Function
    fob = 0
    for i, tgu in enumerate(system.data['thermal-units']):
        fob += tgu["cost"]*g_t[i]
    fob+=TestSystem.data['outage_cost']*deficit[0]
    for i, _ in enumerate(system.data['hydro-units']):
        fob += 0.01*v_v[i]

    ## Hydro Balance
    constraints = []
    for i, hgu in enumerate(system.data['hydro-units']):
        constraints.append( v_f[i] == v_i + inflow - v_t[i] - v_v[i] )

    supplying = 0
    ## Load Supplying
    for i, hgu in enumerate(system.data['hydro-units']):
        supplying += hgu["prod"] * v_t[i]

    for i, tgu in enumerate(system.data['thermal-units']):
        supplying += g_t[i]

    supplying += deficit[0]

    constraints.append(supplying == system.data['load'][2])

    ## Constraints
    for i, hgu in enumerate(system.data['hydro-units']):
        constraints.append(v_f[i] >= hgu["v_min"])
        constraints.append(v_f[i] <= hgu["v_max"])
        constraints.append(v_t[i] >= 0)
        constraints.append(v_t[i] <= hgu["flow_max"])
        constraints.append(v_v[i] >= 0)

    for i, tgu in enumerate(system.data['thermal-units']):
        constraints.append(g_t[i] >= 0)
        constraints.append(g_t[i] <= tgu["capacity"])

    constraints.append(deficit[0] >= 0)
    
    opt_problem = model.op(objective=fob, constraints=constraints)
    opt_problem.solve(format='dense',solver='glpk')

    print("Total Cost: {}".format(fob.value()))

    for i, hgu in enumerate(system.data['hydro-units']):
        print("{} {} is {} hm3".format(v_f.name,i,v_f[i].value()))
        print("{} {} is {} hm3".format(v_t.name,i,v_t[i].value()))
        print("{} {} is {} hm3".format(v_v.name,i,v_v[i].value()))

    for i, tgu in enumerate(system.data['thermal-units']):
        print("{} {} is {} MWmed".format(g_t.name,i,g_t[i].value()))

    print("{} is {} MWmed".format(deficit.name,deficit[0].value()))

    for i, hgu in enumerate(system.data['hydro-units']):
        print("The cost of water at Hydro Unit {} is {} hm3".format(i,constraints[i].multiplier.value))

    print("The Marginal Cost is: {}".format(constraints[n_hgu].multiplier.value))

In [5]:
system = TestSystem
v_i = 20
inflow = 11

dispatch(system=system, v_i=v_i, inflow=inflow)

Total Cost: [ 7.67e+03]

Final Volume of the Hydro Unit 0 is [ 2.00e+01]
 hm3
Turbined Flow of the Hydro Unit 0 is [ 1.10e+01]
 hm3
Shed flow of the Hydro Unit 0 is [ 0.00e+00]
 hm3
Power generated by the Thermal Unit 0 is [ 1.50e+01]
 MWmed
Power generated by the Thermal Unit 1 is [ 1.00e+01]
 MWmed
Power deficit is [ 1.45e+01]
 MWmed
The cost of water at Hydro Unit 0 is [ 4.75e+02]
 hm3
The Marginal Cost is: [-5.00e+02]

GLPK Simplex Optimizer, v4.65
12 rows, 6 columns, 17 non-zeros
      0: obj =   0.000000000e+00 inf =   1.010e+02 (3)
      5: obj =   7.675000000e+03 inf =   0.000e+00 (0)
*     6: obj =   7.675000000e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND


In [9]:
import powersddp

system = powersddp.PowerSystem(path='system.yml')

print("System Load: {}\n"
      "Number of HGUs: {}\n"
      "Number of TGUs: {}".format(system.data['load'],
                                  len(system.data['hydro-units']),
                                  len(system.data['thermal-units'])))

System Load: [50, 50, 50]
Number of HGUs: 1
Number of TGUs: 2


In [1]:
import powersddp

payload = {'load': [50, 50, 50],
           'discretizations': 3,
           'stages': 3,
           'scenarios': 2,
           'outage_cost': 500,
           'hydro-units': [{'name': 'HU1',
                            'v_max': 100,
                            'v_min': 20,
                            'prod': 0.95,
                            'flow_max': 60,
                            'inflow_scenarios': [[23, 16],
                                                 [19, 14],
                                                 [15, 11]]}],
          'thermal-units': [{'name': 'GT1', 'capacity': 15, 'cost': 10},
                            {'name': 'GT2', 'capacity': 10, 'cost': 25}]}

system = powersddp.PowerSystem(data=payload)

print("System Load: {}\n"
      "Number of HGUs: {}\n"
      "Number of TGUs: {}".format(system.data['load'],
                                  len(system.data['hydro-units']),
                                  len(system.data['thermal-units'])))

System Load: [50, 50, 50]
Number of HGUs: 1
Number of TGUs: 2
